In [1]:
%matplotlib inline
%env CUDA_VISIBLE_DEVICES=0
import torch
from torch.utils.data import Dataset, DataLoader
from importlib import reload
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import json
from jdconfig import Config

from logger import *

import learning
reload(learning)
from learning import *
import shutil

import pandas as pd


env: CUDA_VISIBLE_DEVICES=0
Using data [Data and variances]


/home/karenleung/repos/trams/agent_models/agent_model_utils.py:5: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/karenleung/miniconda3/envs/trams/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/karenleung/miniconda3/envs/trams/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/karenleung/miniconda3/envs/trams/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/karenleung/miniconda3/envs/trams/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/karenleung/miniconda3/envs/trams/lib

In [2]:
GPU = 0

In [3]:
# settings and configuration parameters

rnn_params = {'dim' : 32, 'nlayers' : 1, 'proj_dim': 32}
model_params = {'input_dim': 28, 'output_dim': 1, 'latent_dim': 3, 'fc_dim': 8, 'log_sigma_clamp': 3, 'standard_p': True, 'gmm_components': 1, 'y_max': 120, 'log_py_clamp': 0.2}
learning_params = {'batch_size': 128, 'seed': 1, 'max_steps': 100, 'lr': 0.0001, 'kl_anneal': (0.0001, 1.0, 50, 2)}
settings_params = {'cuda': True, 'model_dir':'training/GPU'+str(GPU), 'data_dir': '../data/npz/lanechange_slim.npz', 'GPU': GPU, 'scale': 5}
hyperparameters = {'model': model_params, 'learning': learning_params, 'rnn': rnn_params, 'settings': settings_params}


with open(settings_params['model_dir'] + '/hyperparameters.json', 'w') as fp:
    json.dump(hyperparameters, fp)
params = Config(settings_params['model_dir'] + '/hyperparameters.json')

In [4]:
data_np = np.load(params.settings.data_dir)
N = np.arange(len(data_np['tl']))
np.random.shuffle(N)
train_ratio = .9
train_eval_idx = int(np.floor(train_ratio*len(data_np['tl'])))
train_idx = N[:train_eval_idx]
eval_idx = N[train_eval_idx:]

# train data
train_data = torch.tensor(data_np['data_np'][train_idx,:,:], dtype=torch.float)
train_tl = torch.tensor(data_np['tl'][train_idx], dtype=torch.int)
train_col = torch.tensor(data_np['col'][train_idx], dtype=torch.int)

# eval data
eval_data = torch.tensor(data_np['data_np'][eval_idx,:,:], dtype=torch.float)
eval_tl = torch.tensor(data_np['tl'][eval_idx], dtype=torch.int)
eval_col = torch.tensor(data_np['col'][eval_idx], dtype=torch.int)

kwargs = {'num_workers': 1, 'pin_memory': True} if params.settings.cuda else {}
kwargs

data_mean, data_std = get_mean_var_pytorch(train_data, train_tl, 'slim')
train_loader = torch.utils.data.DataLoader(LaneswapDataset(train_data, train_tl, train_col, (data_mean, data_std), scale=params.settings.scale), batch_size = params.learning.batch_size, shuffle=True, **kwargs)
eval_loader = torch.utils.data.DataLoader(LaneswapDataset(eval_data, eval_tl, eval_col, (data_mean, data_std), scale=params.settings.scale), batch_size = params.learning.batch_size, shuffle=True, **kwargs)

In [6]:
params.settings.model_dir + '/checkpoints'

'training/GPU0/checkpoints'

In [19]:
# THIS WILL DELETE THE FOLDERS AND CHECKPOINTS INSIDE.
shutil.rmtree(params.settings.model_dir + '/checkpoints', ignore_errors=True)

In [20]:
# setup model
model = LaneswapModelLogistic(params)
if params.settings.cuda:
    model.cuda(0)
optimizer = torch.optim.Adam(model.parameters(), lr=params.learning.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
logger = Logger(params.settings.model_dir + '/checkpoints')
train_losses = []
eval_losses = []
j = 0
k = 0
for step in range(params.learning.max_steps):
    mode = 'train'
    model.train()
    train_loss = []
#     model.anneal_parameters(step)
#     logger.scalar_summary('train kl weight', model.kl_weight, step)
    for (i, (x, y)) in enumerate(train_loader):
        if params.settings.cuda:
            x['data'] = x['data'].cuda()
            x['tl'] = x['tl'].cuda()
            x['tl_max'] = x['tl_max'].cuda()
            x['collision'] = x['collision'].cuda()
            y = y.cuda()
#             model.kl_weight = model.kl_weight.cuda()
        c, (loss, ll) = model(x, y, mode)
#         print('ll: ',ll.cpu().data.numpy(), '      kl: ', kl.cpu().data.numpy())
        # logging
        logger.scalar_summary('train loss', loss, j)
        logger.scalar_summary('train log likelihood', ll, j)
#         logger.scalar_summary('train kl divergence', kl, j)
        logger.histo_summary('train p_y', c.cpu().data.numpy(), j)
#         logger.histo_summary('train time to collision', mus.cpu().data.numpy(), j)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lossi = loss.cpu().data.numpy()
        train_loss.append(lossi)
        j += 1
    train_loss = np.mean(train_loss)
    train_losses.append(train_loss)
    # evalulation/validation
    mode = 'eval'
    model.eval()
    eval_loss = []
    for (i, (x, y)) in enumerate(eval_loader):
        if params.settings.cuda:
            x['data'] = x['data'].cuda()
            x['tl'] = x['tl'].cuda()
            x['tl_max'] = x['tl_max'].cuda()
            x['collision'] = x['collision'].cuda()
            y = y.cuda()
        c, (loss, ll) = model(x, y, mode)
        # logging
        logger.scalar_summary('eval loss', loss, k)
        logger.scalar_summary('eval log likelihood', ll, k)
#         logger.scalar_summary('eval kl divergence', kl, k)
        logger.histo_summary('eval p_y', c.cpu().data.numpy(), j)
#         logger.histo_summary('eval time to collision', mus.cpu().data.numpy(), k)
        lossi = loss.cpu().data.numpy()
        eval_loss.append(lossi)
        k += 1
    eval_loss = np.mean(eval_loss)
    eval_losses.append(eval_loss)
    
    scheduler.step(eval_loss)
    print('STEP: ', step, ' ---- TRAIN LOSS : ', train_loss, '---- EVAL LOSS', eval_loss)
        

/home/karenleung/miniconda3/envs/trams/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


STEP:  0  ---- TRAIN LOSS :  -0.0797797 ---- EVAL LOSS 0.699682
STEP:  1  ---- TRAIN LOSS :  -0.0857939 ---- EVAL LOSS 0.696258
STEP:  2  ---- TRAIN LOSS :  -0.0913488 ---- EVAL LOSS 0.691304
STEP:  3  ---- TRAIN LOSS :  -0.0966506 ---- EVAL LOSS 0.68675
STEP:  4  ---- TRAIN LOSS :  -0.102014 ---- EVAL LOSS 0.680639
STEP:  5  ---- TRAIN LOSS :  -0.107104 ---- EVAL LOSS 0.677294
STEP:  6  ---- TRAIN LOSS :  -0.112507 ---- EVAL LOSS 0.676699
STEP:  7  ---- TRAIN LOSS :  -0.117551 ---- EVAL LOSS 0.670111
STEP:  8  ---- TRAIN LOSS :  -0.122361 ---- EVAL LOSS 0.670278
STEP:  9  ---- TRAIN LOSS :  -0.126558 ---- EVAL LOSS 0.666299
STEP:  10  ---- TRAIN LOSS :  -0.130391 ---- EVAL LOSS 0.654397
STEP:  11  ---- TRAIN LOSS :  -0.133957 ---- EVAL LOSS 0.658963
STEP:  12  ---- TRAIN LOSS :  -0.136447 ---- EVAL LOSS 0.650801
STEP:  13  ---- TRAIN LOSS :  -0.138659 ---- EVAL LOSS 0.65329
STEP:  14  ---- TRAIN LOSS :  -0.140092 ---- EVAL LOSS 0.647975
STEP:  15  ---- TRAIN LOSS :  -0.141102 ---- EVA

In [42]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
test_loader = torch.utils.data.DataLoader(LaneswapDataset(eval_data, eval_tl, eval_col, (data_mean, data_std), scale=2), batch_size = len(eval_tl), shuffle=True, **kwargs)

# eval_loader = torch.utils.data.DataLoader(LaneswapDataset(eval_data, eval_tl, eval_col, (data_mean, data_std), scale=2), batch_size = params.learning.batch_size, shuffle=True, **kwargs)

mode = 'eval'
model.eval()
eval_loss = []
for (i, (x, y)) in enumerate(test_loader):
    if params.settings.cuda:
        x['data'] = x['data'].cuda()
        x['tl'] = x['tl'].cuda()
        x['tl_max'] = x['tl_max'].cuda()
        x['collision'] = x['collision'].cuda()
        y = y.cuda()
    rnn_output = model.rnn_encoder(x)
    p = model.logistic_projection(rnn_output)
    c = torch.sigmoid(p)
#     c, (loss, ll) = model(x, y, mode)
#     mu, sigma = model.encoder(x, y, mode)
#     z = model.reparameterize(mu, sigma)
#     p_y = model.decoder(z)
    break